# Ipyreact walkthrough

Welcome to this ipyreact walkthrough!   
The tutorial will be based on a very simple react button to show all the ipyreact features.  

**Content** 
* use the %react cell magic
* write a widget
* style this with CSS
* add parameters to your widgets (traitlets)
* interact with these parameters
* simple traitlet oberservation using `change`
* observe a traitlet and call python function
* observe a traitlet and call JavaScript function
* call python functions from JavaScript
* loading components from external files
* enable hot reloading  
* enable autocompletion in IDEs
* print a message at class initialization

First, we will use the **`%react` magic**  from ipyreact.  
The following line registers the cellmagic:

In [ ]:
%pip install -q ipyreact
# This line is for JupyterLite (if this takes more than 10 seconds, something probably hung, restart the kernel and run this cell again)

In [ ]:
%load_ext ipyreact

In [ ]:
%%react

import * as React from "react";

export default function MyButton() {
    return ( < button > X < /button>);
}

Great, here we can see react code rendering in the jupyter notebook!  
Next, we **convert this into a widget.**  
For that, we need the code in a `_esm` string inside a class that inherits from `ipyreact.ReactWidget`.  
`esm` is short for for EcmaScript module, and thats standard for structuring JavaScript code in reusable components.

In [ ]:
import ipyreact

class MyExampleWidget(ipyreact.ReactWidget):
    _esm = """
    import * as React from "react";

    export default function MyButton() {
        return < button > X < /button> 
    };"""

MyExampleWidget()

Let's **style this with CSS**.

In [ ]:
from IPython.display import HTML
css_rules = """
button {
    background-color: yellow;
    border-radius: 10px;
}
"""
HTML("<style>" + css_rules + "</style>")

Note: Loading CSS styles is global and will include every cell of the notebook.   
Also the widgets at the bottom of the notebook will be affected.  

You can also load CSS from a file.  
And you can add extra CSS without overwriting the previous CSS import.

In [ ]:
from IPython.display import HTML
from pathlib import Path
css_rules = Path("styles_orange.css").read_text()
HTML("<style>" + css_rules + "</style>")

Note: When you clear the output of the previous cell, this CSS style will be removed from the notebook.

Next, we want to **add parameters to the widget.**  
First naive approach: Using f-strings.
That works, but is bad for two reasons:  
1. curly brackets from f-strings will collide with the curly brackets from JavaScript/TypeScript. 
Using curly brackets in an f-string as a normal character can be done by doubling them like this : `{`to `{{`.  
2. The f-string is only interpreted when the class is interpreted. That means it's won't be possible to change the value of `my_message` when interacting with the widget.
   

In [ ]:
import ipyreact

# ❌❌❌  WARNING: THIS CODEBLOCK IS NOT GOOD PRACTICE ❌❌❌ 
class MyExampleWidget(ipyreact.ReactWidget):
    my_message = "Hello World"
    _esm = f"""
    import * as React from "react";

    export default function MyButton() {{
        return <button> {my_message} </button> 
        }};"""
MyExampleWidget()

Instead, we can **use [traitlets](https://traitlets.readthedocs.io/en/stable/using_traitlets.html)**.  
Traitlets are objects that can be used both in JavaScript and python.  
Traitlets that are tagged with sync=True will be available not only on the Python side, but also in the frontend.  

To access the value from the JavaScript side, we can use that all values are passed as a single object as first argument.  
So we can use [object destructuring](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Operators/Destructuring_assignment#object_destructuring) to get access to the variable: `MyButton({my_message})`.

In [ ]:
import ipyreact
from traitlets import Unicode

# ⭐⭐⭐  This is good practice again ⭐⭐⭐
class MyExampleWidget(ipyreact.ReactWidget):
    my_message = Unicode("Hi There").tag(sync=True)
    _esm = """
    import * as React from "react";

    export default function MyButton({ my_message }) {
    return <button> {my_message} </button>;
    };"""
MyExampleWidget()

In [ ]:
# we can use this traitlets also as parameters
MyExampleWidget(my_message="Super Button")

Here we have an example of a button that changes the postion using traitlets

In [ ]:
import ipyreact
from traitlets import Int

class MyExampleWidget(ipyreact.ReactWidget):
    my_width = Int(23).tag(sync=True)
    _esm = """
    import * as React from "react";

    export default function MyButton({ my_width }) {
      return (
        <button
          style={{
            position: "relative",
            width: my_width,
            height: 30,
          }}
        >
          {" "}
          Width of {my_width} px{" "}
        </button>
      );
    }"""
MyExampleWidget(my_width=300)

In [ ]:
MyExampleWidget(my_width=600)
#MyExampleWidget(my_width="Hello") # <- this will throw an error, as my_width expects an integer

It's great that python will throw an error when the wrong type is given!

In [ ]:
# you can also create an instance of a class
w = MyExampleWidget(my_width=600)
w

In [ ]:
# and display this instance multiple times
w

In [ ]:
w.my_width = 400 # see how this will change the button width

In [ ]:
w.my_width = 200 # and one more time

In [ ]:
w.my_width # and get current parameter like this

Next, we will **add a counter** to this widget.

There are two options:
🌱 Firstly, we can define a `my_count` traitlet and for the type we choose an Int traitlet.  
Note that for for every trait <name> there is a `<name> + set_<name>` pair on the react/frontend side.  
That means for `my_count`, there is `set_my_count`.  


In [ ]:
import ipyreact
from traitlets import Int

# 🌱🌱🌱 First option 🌱🌱🌱


class MyCounterWidget(ipyreact.ReactWidget):
    my_count = Int(0).tag(sync=True)
    _esm = """
    import * as React from "react";

    export default function MyButton({my_count, set_my_count}) {
        return <button onClick={() => set_my_count(my_count + 1)}>
            {my_count} times clicked
        </button>
    };"""
MyCounterWidget()

🍁 Secondly, we can use the build in `value` traitlet, which has the type of traitlets.Any type.
And for `value` there is `set_value`.  


In [ ]:
import ipyreact

# 🍁🍁🍁  Second option 🍁🍁🍁


class MyCounterWidget(ipyreact.ReactWidget):
    _esm = """
    import * as React from "react";

    export default function MyButton({value, set_value}) {
        return <button onClick={() => set_value(value + 1)}>
            {value || 0} times clicked
        </button>
    };"""
m = MyCounterWidget()
m

In [ ]:
m.value # <- like this you can access the value from python

In [ ]:
m.value = 20 # <- setting the value also works here.

next, let's make a **simple traitlet oberservation using `change`.**   
Note that `self.count` and `change['new']` is the same.  

In [ ]:
from traitlets import  Any, observe
from traitlets import Int, Any
import ipyreact

class Example(ipyreact.ReactWidget):
    count = Int(0).tag(sync=True)

    @observe("count")
    def _observe_count(self, change):
        print(f"Old value: {change['old']}")
        print(f"New value: {change['new']}")
        print(f"--------------------------")

    _esm = """
    import * as React from "react";

    export default function({set_count, count, prime_message}) {
        return <div><button onClick={() => set_count(count + 1)}>
            {count} times clicked 
        </button>
        </div>
    };"""


Example()

next, let's **observe a traitlet and call python function** everytime the value of that traitlet changes.  
First we need a python function, e.g. in our example it will tell us if we have a prime number or not:

In [ ]:
from traitlets import  Any, observe
from traitlets import Int, Any
import ipyreact

def is_prime_number(n):
    for i in range(2, n):
        if n % i == 0:
            return "No 💻🧊🧊🧊"
    return "Yes 💻✅✅✅"


class PrimePythonWidget(ipyreact.ReactWidget):
    prime_message = Any("Click the Button").tag(sync=True)
    count = Int(0).tag(sync=True)

    @observe("count")
    def _observe_count(self, change):
        self.prime_message = is_prime_number(self.count)

    _esm = """
    import * as React from "react";

    export default function({set_count, count, prime_message}) {
        return <div><button onClick={() => set_count(count + 1)}>
            {count} times clicked 
        </button>
        <br/>
        <span> {prime_message} </span>
        </div>
    };"""


primepy = PrimePythonWidget()
primepy

In [ ]:
primepy.count = 3

In [ ]:
primepy.count = 4

Cool! But is this also possible to  **observe a traitlet and call Javascript function** everytime the value of that traitlet changes?   
Indeed, it is.  (TODO: This example is not yet ready!)
Like this:

In [ ]:
class PrimeJavaScriptWidget(ipyreact.ReactWidget):
    prime_message = Any("Click the Button").tag(sync=True)    # <- TODO: this message does not show up because prime_message is overwritten
    count = Int(0).tag(sync=True)

    _esm = """
    import * as React from "react";

    function isPrimeNumber(n) {
    for (let i = 2; i < n; i++) {
        if (n % i === 0) {
        return "No 🌐🧊🧊🧊";
        }
    }
    return "Yes 🌐✅✅✅";
    }

    export const MyUpdater = ({ count, prime_message}) => {
    prime_message = isPrimeNumber(count);
    return <span> {prime_message} </span>;
    };

    export default function ({ set_count, count, prime_message}) {
    return (
        <div>
        <button onClick={() => set_count(count + 1)}>{count} times clicked</button>
        <br />
        <MyUpdater count={count} prime_message = {prime_message}/>
        </div>
    );
    }
    """
primejs = PrimeJavaScriptWidget()
primejs

It is also be possible to **call python functions from JavaScript**  like this.  
Here we need the on_ prefix for the function name.s

In [ ]:
import ipyreact
from traitlets import Int, Unicode

class Widget1(ipyreact.ReactWidget):
    my_count = Int(0).tag(sync=True)
    label = Unicode("Click me").tag(sync=True)

    def on_my_python_function(self):
        self.my_count = self.my_count + 1
        self.label = f"Clicked {self.my_count}"

    _esm = """
        import * as React from "react";

        export default function({on_my_python_function, label}) {
            return(
            <div>
                <button onClick={() => on_my_python_function()}>
                    {label}
                </button>
            </div>
            )
        };
    """
            
w1 = Widget1()
w1

Having the JavaScript components in python string variables is good for the beginning.  
That way the project is compact and there is no need of file switching.  

As examples are getting longer, the JavaScript components can be written in separate files.  
That way, you will also get JavaScript syntax hilighting.  

In [ ]:
import ipyreact
import pathlib

class WidgetFromFile(ipyreact.ReactWidget):
    _esm = pathlib.Path("my_component.tsx").read_text()

WidgetFromFile()

If you don't want to re-run the python code after making changes to the file that contains the component, you can see changes happening immediately thanks to **hot-reloading**.
This requires `pip install watchdog`.  
Next, you replace the line  
`_esm = pathlib.Path("my_component.tsx").read_text()`  
with   
`_esm = pathlib.Path("my_component.tsx")`  

Now open `my_component.tsx`, change "Hello World" to "Hi there", and you will see that the changes are reflected immediately.

In [ ]:
import ipyreact
import pathlib

class WidgetFromFile(ipyreact.ReactWidget):
    _esm = pathlib.Path("my_component.tsx") # <- this will not work in JupyterLite

WidgetFromFile()

one more thing:  
Having **autocompletion in IDEs** is awesome!  
traitlets don't have that by default, but adding a `signature_has_traits` decorator will do the job!

In [ ]:
import ipyreact
from traitlets import Any, Unicode, Int, observe, signature_has_traits

@signature_has_traits
class MyExampleWidget(ipyreact.ReactWidget):
    my_width = Int(23).tag(sync=True)
    _esm = """
    import * as React from "react";

    export default function MyButton({ my_width }) {
      return (
        <button
          style={{
            position: "relative",
            width: my_width,
            height: 30,
          }}
        >
          {" "}
          Width of {my_width} px{" "}
        </button>
      );
    }"""
MyExampleWidget(my_width=300)

And this screenshots shows that autocompletion works now:  
<img src="autocomplete_screenshot.png" width="400">

Now we want to **print a message at class initialization** that says "Button was initialized with width 300px!"  
That is possible with the following code pattern using calling the `super` method.

In [ ]:
import ipyreact
from traitlets import Int, signature_has_traits

# 🪄🪄🪄 this is an advanced example, feel free to skip 🪄🪄🪄

@signature_has_traits
class MyExampleWidget(ipyreact.ReactWidget):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.print_welcome_message()

    def print_welcome_message(self):
        print(f"Button was initilized with width of {self.my_width}px ")

    my_width = Int(23).tag(sync=True)

    _esm = """
    import * as React from "react";

    export default function MyButton({ my_width }) {
    return (
        <button
        style={{
            position: "relative",
            width: my_width,
            height: 30,
        }}
        >
        {" "}
        Width of {my_width} px{" "}
        </button>
    );
    }"""

MyExampleWidget(my_width=200)